In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim



In [22]:
# Load the wine dataset
data_columns = [
    "Class", "Alcohol", "Malic_acid", "Ash", "Alcalinity_of_ash",
    "Magnesium", "Total_phenols", "Flavanoids", "Nonflavanoid_phenols",
    "Proanthocyanins", "Color_intensity", "Hue", "OD280/OD315", "Proline"
]

wine_data = pd.read_csv("wine.data", header=None, names=data_columns)

In [23]:
# Split features and target
X = wine_data.drop("Class", axis=1)
y = wine_data["Class"]


In [24]:
# One-hot encode the target
ohe = OneHotEncoder(sparse_output=False)  # Ganti 'sparse' dengan 'sparse_output'
y_encoded = ohe.fit_transform(y.values.reshape(-1, 1))

In [25]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [26]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [28]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, activation_fn):
        super(MLP, self).__init__()
        layers = []
        activation_map = {
            'linear': nn.Identity(),
            'sigmoid': nn.Sigmoid(),
            'relu': nn.ReLU(),
            'softmax': nn.Softmax(dim=1),
            'tanh': nn.Tanh()
        }
        for i, hidden_size in enumerate(hidden_layers):
            layers.append(nn.Linear(input_size if i == 0 else hidden_layers[i-1], hidden_size))
            layers.append(activation_map[activation_fn])
        layers.append(nn.Linear(hidden_layers[-1], output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [34]:
# Experiment function
def run_experiment(hidden_layers, activation_fn, epochs, learning_rate, batch_size):
    input_size = X_train.shape[1]
    output_size = y_train.shape[1]

    model = MLP(input_size, hidden_layers, output_size, activation_fn)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(X_train_tensor.size(0))
        for i in range(0, X_train_tensor.size(0), batch_size):
            indices = permutation[i:i + batch_size]
            batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.argmax(dim=1))
            loss.backward()
            optimizer.step()
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        _, actual = torch.max(y_test_tensor, 1)
        accuracy = (predicted == actual).sum().item() / y_test_tensor.size(0)
    return accuracy



In [41]:
# Experiment: Hidden Layers
hidden_layer_configs = [[4], [8], [16], [32], [64], [4, 4], [8, 8], [16, 16], [32, 32], [64, 64]]
print("Hidden Layers Experiment")
for hidden_layers in hidden_layer_configs:
    accuracy = run_experiment(hidden_layers, 'relu', 50, 0.01, 32)
    print(f"Hidden Layers: {hidden_layers}, Accuracy: {accuracy:.2f}")

Hidden Layers Experiment
Hidden Layers: [4], Accuracy: 1.00
Hidden Layers: [8], Accuracy: 1.00
Hidden Layers: [16], Accuracy: 1.00
Hidden Layers: [32], Accuracy: 1.00
Hidden Layers: [64], Accuracy: 1.00
Hidden Layers: [4, 4], Accuracy: 1.00
Hidden Layers: [8, 8], Accuracy: 1.00
Hidden Layers: [16, 16], Accuracy: 1.00
Hidden Layers: [32, 32], Accuracy: 1.00
Hidden Layers: [64, 64], Accuracy: 1.00


In [42]:
# Experiment: Activation Functions
activation_functions = ['linear', 'sigmoid', 'relu', 'softmax', 'tanh']
print("\nActivation Functions Experiment")
for activation_fn in activation_functions:
    accuracy = run_experiment([32], activation_fn, 50, 0.01, 32)
    print(f"Activation Function: {activation_fn}, Accuracy: {accuracy:.2f}")



Activation Functions Experiment
Activation Function: linear, Accuracy: 1.00
Activation Function: sigmoid, Accuracy: 1.00
Activation Function: relu, Accuracy: 1.00
Activation Function: softmax, Accuracy: 1.00
Activation Function: tanh, Accuracy: 1.00


In [43]:
# Experiment: Epochs
epochs_list = [1, 10, 25, 50, 100, 250]
print("\nEpochs Experiment")
for epochs in epochs_list:
    accuracy = run_experiment([32], 'relu', epochs, 0.01, 32)
    print(f"Epochs: {epochs}, Accuracy: {accuracy:.2f}")


Epochs Experiment
Epochs: 1, Accuracy: 0.97
Epochs: 10, Accuracy: 1.00
Epochs: 25, Accuracy: 1.00
Epochs: 50, Accuracy: 1.00
Epochs: 100, Accuracy: 1.00
Epochs: 250, Accuracy: 1.00


In [44]:
# Experiment: Learning Rates
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
print("\nLearning Rates Experiment")
for learning_rate in learning_rates:
    accuracy = run_experiment([32], 'relu', 50, learning_rate, 32)
    print(f"Learning Rate: {learning_rate}, Accuracy: {accuracy:.2f}")


Learning Rates Experiment
Learning Rate: 10, Accuracy: 1.00
Learning Rate: 1, Accuracy: 1.00
Learning Rate: 0.1, Accuracy: 1.00
Learning Rate: 0.01, Accuracy: 1.00
Learning Rate: 0.001, Accuracy: 1.00
Learning Rate: 0.0001, Accuracy: 0.92


In [45]:
# Experiment: Batch Sizes
batch_sizes = [16, 32, 64, 128, 256, 512]
print("\nBatch Sizes Experiment")
for batch_size in batch_sizes:
    accuracy = run_experiment([32], 'relu', 50, 0.01, batch_size)
    print(f"Batch Size: {batch_size}, Accuracy: {accuracy:.2f}")

print("\nAll experiments completed.")



Batch Sizes Experiment
Batch Size: 16, Accuracy: 1.00
Batch Size: 32, Accuracy: 1.00
Batch Size: 64, Accuracy: 1.00
Batch Size: 128, Accuracy: 1.00
Batch Size: 256, Accuracy: 1.00
Batch Size: 512, Accuracy: 1.00

All experiments completed.
